In [3]:
import os
import spotipy
from urllib.request import urlretrieve
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from time import sleep

In [2]:
def spotify_login(cid, secret):
    client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
    return spotipy.Spotify(client_credentials_manager=client_credentials_manager)

cid = "" 
secret = ""



In [4]:
# Defining sub genres
genre_list = ["thrash metal", "death metal", "black metal", "grindcore", "melodic death metal", 
          "power metal", "metalcore", "deathcore", "nwobhm", "doom metal", 
          "nu metal", "folk metal", "blackgaze", "glam metal", "goregrind", 
          "industrial metal", "progressive metal", "djent", "sludge metal",   
          "technical death metal", "funeral doom", "symphonic metal", "drone metal", 
          "slam death metal", "depressive black metal"]

exclusion_lists = {
    'power metal': ['Accept', 'Amon Amarth', 'Children Of Bodom', 'Arch Enemy', 'Amorphis', 'Lordi', 'Equilibrium', 'Machinae Supremacy', 'Metal Church', 'Nevermore', 'X JAPAN', 'Queensrÿche', 'Stryper', 'Mafumafu', 'Dimmu Borgir', 'Imperial Circus Dead Decadence', 'Savatage', 'Saxon', 'Moonspell', "Therion", 'Armored Saint', 'Sanctuary', 'Manilla Road', 'Aldious', 'A Sound of Thunder', 'Asura', 'Bonfire', 'Bruce Dickinson', 'Civil War', 'Dark Sarah', 'Doro', 'Dust in Mind', 'ELFENSJóN', 'Fates Warning', 'Helstar', 'KAMIJO', 'Lizzy Borden', "Mary's Blood", 'Narnia', 'Persuader', 'Rage', 'Quiet Riot', 'The Murder of My Sweet','Unlucky Morpheus', 'U.D.O.', 'Finntroll', 'Elyose', 'Fates Warning', 'Fraser Edwards', 'Heavenly', 'Whitecross', 'Eluveitie'],
    'black metal': ['Vader', 'Dystopia', 'Morbid Angel', 'Deicide', 'Entombed', 'Suffocation', 'Bloodbath', 'Gorguts', 'Bewitcher', 'Hypocrisy', 'Immolation', 'Inferi', 'Cryptopsy', 'Dismember', "Therion", 'Sarcófago', 'Asphyx', 'Unleashed', 'Vital Remains', 'Grave', 'Incantation', 'Sinister', 'Aara', 'Asunojokei', 'Craft', 'Deadlife', 'Grima'],
    'thrash metal': ['Helloween', 'Iced Earth', 'Crimson Glory', 'Jag Panzer', 'Cirith Ungol', 'Vicious Rumors', 'Omen', 'Agent Steel', 'Immortal', 'Bathory', 'Venom', 'Possessed', 'Hellhammer', 'Deströyer 666', 'Wraith', 'Sadistik Exekution', 'Bewitcher', 'Cruel Force', 'Desaster', 'Dark Angel'],
    'nu-metal': ['Lonewolf', 'Begotten', 'Megadeth', 'Slayer', 'Sepultura', 'Suicidal Tendencies', 'Anthrax', 'Cavalera Conspiracy']
}

In [5]:
def get_artists_by_genre(genre, limit=50, pages=10):
    results = []

    for i in range(pages):
        results_page = sp.search(q=f'genre:"{genre}"', type='artist', limit=limit, offset=limit * i)
        results += results_page['artists']['items']

    return results

In [6]:
# Step 3: Create hierarchy for each subgenre

genre_data = {}

def combine_subgenres(genre_data):
    combined_genre_data = {}
    for genre, artists in genre_data.items():
        if genre in ['melodic death metal', 'technical death metal', 'slam death metal']:
            genre = 'death metal'
        elif genre == 'depressive black metal':
            genre = 'black metal'
        combined_genre_data.setdefault(genre, []).extend(artists)
    return combined_genre_data

# Process genre data
for subgenre in genre_list:
    output_dir = os.path.join('data', subgenre.replace(' ', '_'))
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

for genre in genre_list:
    results = get_artists_by_genre(genre)
    genre_data[genre] = [artist['name'] for artist in results if artist['name'] not in exclusion_lists.get(genre, [])]

# Combine subgenres into main genre categories
genre_data = combine_subgenres(genre_data)

# Find the maximum length of the lists
max_length = max(len(artists) for artists in genre_data.values())

# Pad lists with empty strings to ensure they all have the same length
for genre, artists in genre_data.items():
    genre_data[genre] += [''] * (max_length - len(artists))

# Create DataFrame
df = pd.DataFrame(genre_data)
display(df.head())
print(df.shape)

NameError: name 'sp' is not defined

In [ ]:
df.to_csv('bands_by_genre.csv')

In [4]:
import os
from urllib.request import urlretrieve
import pandas as pd

def get_artist_info(artist, genre):
    sp = spotify_login(cid, secret)
    result = sp.search(artist, type='artist')
    # Initialize an empty list to store track information
    track_info_list = []

    result = sp.search(artist, type='artist')
    artist_id = result['artists']['items'][0]['id']

    # Get albums of the artist
    albums = sp.artist_albums(artist_id)

    # Iterate through each album
    for album in albums['items']:
        album_name = album['name']

        # Get tracks for each album
        album_tracks = sp.album_tracks(album['id'])

        # Iterate through each track
        for track in album_tracks['items']:
            track_name = track['name']
            preview_url = track['preview_url']

            # Get audio features for the track
            audio_features = sp.audio_features(track['id'])[0]  # Assuming there's only one track per ID
            if not audio_features:
                continue  # Skip if no audio features are available

            # Save preview track
            if preview_url:
                filename = f"{artist}_{album_name}_{track_name}.mp3"
                filepath = os.path.join( filename)  # Save in the genre folder
                urlretrieve(preview_url, filepath)

            # Append track information to the list
            track_info = {
                'artist': artist,
                'album': album_name,
                'track': track_name,
                'genre': genre,  # Add genre column
                **audio_features  # Unpack audio_features dictionary into separate columns
            }
            track_info_list.append(track_info)

    # Create DataFrame from the list of track information
    df = pd.DataFrame(track_info_list)

    return df
get_artist_info('antzaat','black metal')


In [3]:
def get_discography_info(data):
    big_df = pd.DataFrame()  # Initialize an empty DataFrame to store all track information

    with ThreadPoolExecutor() as executor:
        futures = []  # List to store futures

        # Iterate through each genre column
        for genre, band_names in data.items():
            # Create a folder for the current genre if it doesn't exist
            genre_folder = os.path.join('data/audio_samples', genre)
            os.makedirs(genre_folder, exist_ok=True)

            # Iterate through each band name in the genre column
            for band_name in band_names:
                # Skip if band_name is null
                if pd.isnull(band_name):
                    continue

                # Submitting a task to the executor to fetch track information for each band
                future = executor.submit(get_artist_info, band_name, genre, genre_folder)
                futures.append(future)

        # Iterate through futures to get results
        for future in futures:
            df = future.result()  # Get the DataFrame for each band
            big_df = pd.concat([big_df, df], ignore_index=True)  # Append to big DataFrame

    return big_df


In [7]:
data = pd.read_csv('bands_by_genre.csv')
data = data.iloc[:, 1:]


get_discography_info(data)


Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
